In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
--2019-04-20 14:04:49--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.223, 91.189.89.222, 2001:67c:1560:8003::8003, ...
Connecting to launchpad.net (launchpad.net)|91.189.89.223|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-20 14:04:49 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-drive-ocaml

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
!pip install tensorflow-gpu==2.0.0-alpha0

    100% |████████████████████████████████| 332.1MB 32kB/s 
    100% |████████████████████████████████| 61kB 26.6MB/s 
    100% |████████████████████████████████| 419kB 12.3MB/s 
    100% |████████████████████████████████| 3.0MB 8.7MB/s 


In [0]:
import cv2
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.datasets import load_files
import matplotlib.pyplot as plt

In [0]:
def load_data(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    return files

In [0]:
def ext_img(img):
    imag = cv2.imread(img)
    imag = cv2.cvtColor(imag,cv2.COLOR_BGR2GRAY)
    imag = imag/255
    imag = np.expand_dims(imag,axis=2)
    return imag

In [0]:
def tensor_4d(fil):
    lis = [ext_img(im) for im in tqdm(fil)]
    return np.stack(lis,axis=0)

In [0]:
fold_path = 'drive/datasets/shapes'

In [0]:
img_files = load_data(fold_path)

In [12]:
imgs = tensor_4d(img_files)

100%|██████████| 300/300 [00:01<00:00, 250.16it/s]


In [13]:
imgs.shape

(300, 28, 28, 1)

In [0]:
def Encoder()